<a href="https://colab.research.google.com/github/umleeho1/AI_learning/blob/main/SBERT_SimCSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SimCSE with SBERT

In [ ]:
!pip install sentence-transformers

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

#Check if dataset exsist. If not, download and extract  it
nli_dataset_path = 'datasets/AllNLI.tsv.gz'
sts_dataset_path = 'datasets/stsbenchmark.tsv.gz'

if not os.path.exists(nli_dataset_path):
    util.http_get('https://sbert.net/datasets/AllNLI.tsv.gz', nli_dataset_path)

if not os.path.exists(sts_dataset_path):
    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)


#You can specify any huggingface/transformers pre-trained model here, for example, bert-base-uncased, roberta-base, xlm-roberta-base
model_name = 'distilbert-base-uncased'

model_save_path = 'output/training_simcse-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
word_embedding_model = models.Transformer(model_name, max_seq_length=64)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


2021-04-20 20:22:46 - Use pytorch device: cuda


For SimCSE, we create our train_samples with InputExamples that consists of two times the same sentences, i.e.
```
train_samples = [InputExample(texts=["sent1", "sent1"]), InputExample(texts=["sent2", "sent2"]), ...]
````


In [ ]:
# Read the AllNLI.tsv.gz file and create the training dataset
# We just pass sentence1 two times to InputExample and use MultipleNegativesRankingLoss
logging.info("Read AllNLI train dataset")
train_samples = []
with gzip.open(nli_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['split'] == 'train':
            train_samples.append(InputExample(texts=[row['sentence1'], row['sentence1']]))


#Read STSbenchmark dataset and use it as development set
logging.info("Read STSbenchmark dev dataset")
dev_samples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['split'] == 'dev':
            score = float(row['score']) / 5.0 #Normalize score to range 0 ... 1
            dev_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))

dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=8, name='sts-dev')

2021-04-20 20:22:46 - Read AllNLI train dataset
2021-04-20 20:22:55 - Read STSbenchmark dev dataset


As loss, we use: MultipleNegativesRankingLoss

Here, texts[0] and texts[1] are considered as positive pair, while all others are negatives in a batch

In [ ]:
# Configure the training
train_batch_size = 32
num_epochs = 1

# Use MultipleNegativesRankingLoss for SimCSE
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)


warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

logging.info("Performance before training")
dev_evaluator(model)

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=100,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )


2021-04-20 20:22:55 - Warmup-steps: 2944
2021-04-20 20:22:55 - Performance before training
2021-04-20 20:22:55 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset:
2021-04-20 20:23:02 - Cosine-Similarity :	Pearson: 0.6599	Spearman: 0.6764
2021-04-20 20:23:02 - Manhattan-Distance:	Pearson: 0.6863	Spearman: 0.6930
2021-04-20 20:23:02 - Euclidean-Distance:	Pearson: 0.6860	Spearman: 0.6928
2021-04-20 20:23:02 - Dot-Product-Similarity:	Pearson: 0.3630	Spearman: 0.3564


2021-04-20 20:23:38 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 100 steps:
2021-04-20 20:23:41 - Cosine-Similarity :	Pearson: 0.7153	Spearman: 0.7239
2021-04-20 20:23:41 - Manhattan-Distance:	Pearson: 0.7212	Spearman: 0.7278
2021-04-20 20:23:41 - Euclidean-Distance:	Pearson: 0.7206	Spearman: 0.7276
2021-04-20 20:23:41 - Dot-Product-Similarity:	Pearson: 0.4728	Spearman: 0.4815
2021-04-20 20:23:41 - Save model to output/training_simcse-2021-04-20_20-22-45
2021-04-20 20:24:17 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 200 steps:
2021-04-20 20:24:20 - Cosine-Similarity :	Pearson: 0.7618	Spearman: 0.7592
2021-04-20 20:24:20 - Manhattan-Distance:	Pearson: 0.7497	Spearman: 0.7523
2021-04-20 20:24:20 - Euclidean-Distance:	Pearson: 0.7492	Spearman: 0.7519
2021-04-20 20:24:20 - Dot-Product-Similarity:	Pearson: 0.5892	Spearman: 0.6102
2021-04-20 20:24:20 - Save model to output/training_simcse-2021-04-20_20-

In [ ]:


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

test_samples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['split'] == 'test':
            score = float(row['score']) / 5.0 #Normalize score to range 0 ... 1
            test_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))

model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=train_batch_size, name='sts-test')
test_evaluator(model, output_path=model_save_path)